TBC: To be confirmed

# 0. Inspect data and code

1. Check features
    - ~~inspect visual features~~
    - ~~how they generate the 21 feature maps?~~
    - ~~text features~~
    - ~~Does it  include feature extraction code?~~
    - video metadata
    
2. Run code
    - ~~run test_network.sh~~
    - ~~run run_job_rgb.sh~~
    
3. Batching
    - generation
    
4. Text pre-processing
    - details

## Summary

1. Feature description
    - RGB and flow features are of size 4096 and 1024, respectively.
    - The HDF5 file contains a double tensor of size (6, feat_size)
    - TBC: the value on each matrix corresponds to the feature of a 5s chunk.
    - Features for each of the 21 possible segments and context features are computed on-the-fly.
        - Take a look at L:181:L:197 in file `data_processing.py`
            - [URL]()
        - Normalization: Frobenius norm of the feature matrix
    - Text queries are ...
        - There are two features glove matrix and number of words in query.
        - Matrix of `[max_len_sentence, num_glove_centroids]`
            - Matrix is arrange in a particular way. Order is consistent but short sentences will be padded with a lot of zeros at the begining.
        - Number of words is encoded as a vector of size [max_len_sentence]
            - This is a binary vector with 1s on the locations where the query has words.
    
## Questions

1. Visaul feature details
    1. value on each matrix corresponds to the feature of a 5s chunk. If shorter is the zero vector.
    1. feature extraction details
    1. Frobenius normalization vs avg pooling
1. Loss function
    - Mini-batch generation
1. Code details
    a. Block of code is repeated in L:43-L45 in `data_processing.py`

### 0.1.a inspect visual features

In [ ]:
import h5py
filename = 'data/average_fc7.h5'
fid = h5py.File(filename)
keys = list(fid.keys())
print('Number of keys in file', len(keys))
print('Type of keys', type(fid[keys[0]]))
print('Shape of value', fid[keys[0]].shape)
print('Type of value', fid[keys[0]].dtype)

feat_size = fid[keys[0]].shape
for k, v in fid.items():
    assert all([j == feat_size[i] for i, j in enumerate(v.shape)])